In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import zipfile

# Define the zip file and destination folder
zip_path = r'../data/interim/df_mouse_p1p2.zip'  
extract_folder = "../data/interim/unzipped/"  

# Unzip the folder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracted to {extract_folder}")

# Define the zip file and destination folder
zip_path = r'../data/interim/weblog_p1p2.zip'  
extract_folder = "../data/interim/unzipped/"  

# Unzip the folder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracted to {extract_folder}")

# Unzipped data will be gitignored

Extracted to ../data/interim/unzipped/
Extracted to ../data/interim/unzipped/


In [10]:
# Load the data
file_path = "../data/interim/unzipped/df_mouse_p1p2.csv"
df = pd.read_csv(file_path)

# Drop duplicates, drop NAs, and sort values
df = df.drop_duplicates(subset=['session_id', 'mousemove_times'], keep='first')
df = df.sort_values(by=['session_id', 'mousemove_times']).reset_index(drop=True)
df = df.dropna(subset=['session_id', 'mousemove_times', 'mousemove_type', 'mousemove_behaviour', 'label'])

# Extract the mousemove_x and mousemove_y coordinates from the 'mousemove_behaviour' string column
df[['mousemove_x', 'mousemove_y']] = df['mousemove_behaviour'].str.replace(r'[()]', '', regex=True).str.split(',', expand=True)

# Convert mousemove_x and mousemove_y to integer type
df['mousemove_x'] = df['mousemove_x'].astype(int)
df['mousemove_y'] = df['mousemove_y'].astype(int)

# Convert mousemove_type to categorical type
df = pd.get_dummies(df, columns=['mousemove_type'], dtype=int)

df

/var/folders/y4/lw6k9pr97v32ftq68tmqp5w00000gn/T/ipykernel_58360/499077550.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,session_id,mousemove_times,mousemove_behaviour,label,mousemove_client_height_width,mousemove_x,mousemove_y,mousemove_type_c,mousemove_type_cl,mousemove_type_cm,mousemove_type_cr,mousemove_type_cu,mousemove_type_m
0,03jt2p4bdru20sjb9me2gco6j4,212340127,"(115,497)",human,"(975, 1920)",115,497,0,1,0,0,0,0
1,03jt2p4bdru20sjb9me2gco6j4,212650194,"(121,314)",human,"(975, 1903)",121,314,0,1,0,0,0,0
2,03jt2p4bdru20sjb9me2gco6j4,212756263,"(77,219)",human,"(975, 1920)",77,219,0,1,0,0,0,0
3,03jt2p4bdru20sjb9me2gco6j4,212822729,"(276,744)",human,"(975, 1903)",276,744,0,1,0,0,0,0
4,03jt2p4bdru20sjb9me2gco6j4,212848259,"(118,196)",human,"(975, 1920)",118,196,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3639211,vu3fio88psda005g91fbjona0v,1606295812254,"(81,573)",moderate_bot,"(847, 1813)",81,573,0,0,0,0,0,1
3639212,vu3fio88psda005g91fbjona0v,1606295812272,"(80,573)",moderate_bot,"(847, 1813)",80,573,0,0,0,0,0,1
3639213,vu3fio88psda005g91fbjona0v,1606295812288,"(79,573)",moderate_bot,"(847, 1813)",79,573,0,0,0,0,0,1
3639214,vu3fio88psda005g91fbjona0v,1606295812305,"(78,574)",moderate_bot,"(847, 1813)",78,574,0,0,0,0,0,1


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Select relevant columns for logistic classification
df_lc = df[['mousemove_x', 'mousemove_y', 'mousemove_type_c', 'mousemove_type_cl', 'mousemove_type_cm', 'mousemove_type_cr', 'mousemove_type_cu', 'mousemove_type_m', 'label']]

# Features and target
X = df_lc.drop(columns='label')
y = df_lc['label']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit logistic regression model
clf = LogisticRegression(max_iter=1)
clf.fit(X_train, y_train)

# Predict class labels for test set
y_pred = clf.predict(X_test)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# Optional: Detailed metrics for evaluation
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

/Users/E0775244/DSA4263-automated-fraud/venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy: 0.4245228956492701


/Users/E0775244/DSA4263-automated-fraud/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/E0775244/DSA4263-automated-fraud/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/E0775244/DSA4263-automated-fraud/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,


Classification Report:
               precision    recall  f1-score   support

advanced_bot       0.42      1.00      0.60    308913
       human       0.00      0.00      0.00    231648
moderate_bot       0.00      0.00      0.00    187110

    accuracy                           0.42    727671
   macro avg       0.14      0.33      0.20    727671
weighted avg       0.18      0.42      0.25    727671

Confusion Matrix:
 [[308913      0      0]
 [231648      0      0]
 [187110      0      0]]
